In [1]:
import os
import re
import json
import tokenizers
import transformers

import pandas as pd

from tqdm import tqdm
from pathlib import Path

In [2]:
def basicPreprocess(text):
    processed_text = text.lower()
    processed_text = re.sub(r'\W +', ' ', processed_text)
    return processed_text

In [3]:
complete_df = pd.read_csv("data/clean_df.csv")

In [4]:
data = complete_df.sample(frac = 1).sample(frac = 1)

In [5]:
data.dropna(inplace = True)

In [6]:
data = data["abstract"].apply(basicPreprocess).replace("\n"," ")

In [7]:
txt_files_dir = "text_splits"

In [8]:
i = 0
for row in tqdm(data.to_list()):
    file_name = os.path.join(txt_files_dir, str(i)+'.txt')
    f = open(file_name, 'w')
    f.write(row)
    f.close()
    i += 1

100%|██████████| 40152/40152 [00:05<00:00, 6775.52it/s]


## Initialize Tokenizer

In [9]:
paths = [str(x) for x in Path(txt_files_dir).glob("**/*.txt")]

tokenizer = tokenizers.ByteLevelBPETokenizer()

vocab_size = 5000

tokenizer.train(files = paths, vocab_size = vocab_size, min_frequency = 5, special_tokens = [
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

## Train Test Split

In [10]:
lm_data_dir = "lm_data"

In [11]:
train_split = 0.9
train_data_size = int(len(data)*train_split)

with open(os.path.join(lm_data_dir,'train.txt') , 'w') as f:
    for item in data[:train_data_size].tolist():
        f.write("%s\n" % item)

with open(os.path.join(lm_data_dir,'eval.txt') , 'w') as f:
    for item in data[train_data_size:].tolist():
        f.write("%s\n" % item)

In [12]:
tokenizer.save("models/COVID", "COVID-BERT")

['models/COVID/COVID-BERT-vocab.json', 'models/COVID/COVID-BERT-merges.txt']

In [13]:
train_path = os.path.join(lm_data_dir,"train.txt")
eval_path = os.path.join(lm_data_dir,"eval.txt")

## Init Config

In [14]:
config = {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.3,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "num_attention_heads": 1,
  "num_hidden_layers": 1,
  "vocab_size": vocab_size,
  "intermediate_size": 256,
  "max_position_embeddings": 256
}

with open("models/COVID/config.json", 'w') as fp:
    json.dump(config, fp)